In [55]:
######数据预处理
#读取数据
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import pickle
import csv
import numpy as np 

w2id , id2w = pickle.load(open('data/w2id_id2w.pkl' , 'rb'))  #词汇表
poem = pickle.load(open('data/index.pkl' , 'rb'))
classified_csvpoem = open("data/Classified_Poem.csv", "r", encoding="utf-8-sig")  #分类过的诗的CSV文件
reader = csv.reader(classified_csvpoem)

poem_list = []   #存放诗词的数组
label_list = []  #存放春夏秋冬的标签的数组
for line in reader:  
    poem_list.append(line[:-1])
    label_list.append(line[33])
    
print ('唐诗数据集大小 : {}'.format(len(poem_list)))
print(len(w2id))

# print(poem_list[:10])
# print(np.array(poem_list).shape)
# print(label_list[:10])
# print(np.array(label_list).shape)



唐诗数据集大小 : 232670
5295


In [56]:
#打乱数据
import random
seed = 50
random.seed(seed)
random.shuffle(poem_list)
random.seed(seed) #两个list同步打乱
random.shuffle(label_list)


In [57]:
######构建网络
import tensorflow as tf
#定义模型
class TrainModel(object):
    def __init__(self, data, classes, labels, emb_keep, rnn_keep, hidden_size,embed_dim,num_layers, 
                 batch_size, learning_rate, lr_decay_step, lr_decay, max_grad):
        
        self.data = data
        self.classes = classes
        self.labels = labels
        self.emb_keep = emb_keep
        self.rnn_keep = rnn_keep
        self.hidden_size = hidden_size
        self.embed_dim = embed_dim
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.lr_decay = lr_decay
        self.lr_decay_step = lr_decay_step
        self.max_grad = max_grad
        self.vocab_size = 5295
        self.global_step
        self.predict
        self.loss
        self.optimize
    
#     def cell(self): 
        
#         return cell
    
    def predict(self):  #前向传播
        # 创建词嵌入矩阵权重
        embedding = tf.Variable(tf.random_uniform([self.vocab_size, self.embed_dim],-1,1), name="embedding")
        emb = tf.nn.embedding_lookup(embedding, self.data) #词嵌入过程
        emb_dropout = tf.nn.dropout(emb, self.emb_keep)
        #构建RNN单元
        lstm_cell = [tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(self.hidden_size), output_keep_prob=self.rnn_keep) for _ in range(self.num_layers)]
        cell = tf.nn.rnn_cell.MultiRNNCell(lstm_cell)
        #创建softmax层
        softmax_weights = tf.Variable(tf.random_uniform([self.hidden_size, self.vocab_size],-1,1), name="softmax_weights")
        softmax_bais = tf.Variable(tf.random_uniform([self.vocab_size],-1,1), name="softmax_bais")
        #计算循环神经网络的输出
        self.init_state = cell.zero_state(self.batch_size, dtype=tf.float32)
        outputs, last_state = tf.nn.dynamic_rnn(cell, emb_dropout, dtype=tf.float32, initial_state=self.init_state)
        outputs = tf.reshape(outputs, [-1, self.hidden_size])
        # 计算logits
        logits = tf.matmul(outputs, softmax_weights) + softmax_bais
        return logits
        
    #定义损失
    def loss(self):
         # 计算交叉熵
        outputs_target = tf.reshape(self.labels, [-1])
        loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.predict(), labels=outputs_target, )
        # 平均
        cost = tf.reduce_mean(loss)
        return cost
    
    def global_step(self):
        #记录步数
        global_step = tf.Variable(0, trainable=False)
        return global_step
    
    def optimize(self):
        # 学习率衰减
        learn_rate = tf.train.exponential_decay(self.learning_rate, self.global_step, self.lr_decay_step, self.lr_decay)
        # 计算梯度，并防止梯度爆炸
        trainable_variables = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.loss, trainable_variables), self.max_grad)
        # 创建优化器，进行反向传播
        optimizer = tf.train.AdamOptimizer(learn_rate)
        train_op = optimizer.apply_gradients(zip(grads, trainable_variables), self.global_step)
        return train_op
    
    def prob(self):  #计算概率
        probs = tf.nn.softmax(self.predict)
        return probs


In [58]:
#######组织数据集
import numpy as np 

class Dataset(object):
    def __init__(self , batch_size):
        self.batch_size = batch_size
        self.data, self.target = self.read_data()
        self.start = 0
        self.lenth = len(self.data)  #一共有多少首诗
        
    def read_data(self):
        id_list = []
        for poem in poem_list:
            id_list.append([w2id[idx] for idx in poem])
        batch_num = len(id_list) // self.batch_size
        # data和target
        x_data = []
        y_data = []
        # 生成batch
        for i in range(batch_num):
            # 截取一个batch的数据
            start = i * self.batch_size
            end = start + self.batch_size
            batch = id_list[start:end]
            # 计算最大长度
            max_lenth = max(map(len, batch))
            # 填充
            tmp_x = np.full((self.batch_size, max_lenth), 0, dtype=np.int32)
            # 数据覆盖
            for row in range(self.batch_size):
                tmp_x[row, :len(batch[row])] = batch[row]  #矩阵{3635,64,33}
            tmp_y = np.copy(tmp_x)
            tmp_y[:, :-1] = tmp_y[:, 1:]  #data与target错位一个
            x_data.append(tmp_x)
            y_data.append(tmp_y)
#         print(np.array(x_data).shape)
        return x_data, y_data   #所有数据
    
    def next_batch(self):
        start = self.start
        self.start += 1  #下一个batch
        if self.start >= self.lenth:
            self.start = 0
        print(np.array(self.data[start]).shape)
        return self.data[start], self.target[start]
    
# dataset = Dataset(64)
# # dataset.read_data()
# x , y = dataset.next_batch()
# print(x)

(64, 33)
[[4702   10 4267 ...    2    2    2]
 [2697 2158 4073 ...    2    2    2]
 [  33   66 1408 ... 2980 5001    1]
 ...
 [2786 2229 3748 ...    2    2    2]
 [2754 2629 1449 ... 1457 5001    1]
 [1656 1656 2956 ... 2754 5001    1]]


In [59]:
#训练模型
import tensorflow as tf

tf.reset_default_graph() 
TRAIN_TIMES = 30000  # 迭代总次数（没有计算epoch）
SHOW_STEP = 1  # 显示loss频率
SAVE_STEP = 100  # 保存模型参数频率
BATCH_SIZE = 64
CKPT_PATH = 'runs/model_ckpt'  # 模型保存路径

EMB_KEEP = 0.5  # embedding层dropout保留率
RNN_KEEP = 0.5  # lstm层dropout保留率

g = tf.Graph()
with g.as_default():
    x_data = tf.placeholder(tf.int32, [BATCH_SIZE, None])  # 输入数据
    y_data = tf.placeholder(tf.int32, [BATCH_SIZE, None])  # 标签
    emb_keep = tf.placeholder(tf.float32)  # embedding层dropout保留率
    rnn_keep = tf.placeholder(tf.float32)  # lstm层dropout保留率

    data = Dataset(BATCH_SIZE)  # 创建数据集

    model = TrainModel(x_data, label_list, y_data, emb_keep, rnn_keep, 512 , 200 , 2 , BATCH_SIZE , 0.0001 , 600 , 0.92 , 5.0)  # 创建训练模型
    # def __init__(self, data, classes, labels, emb_keep, rnn_keep, hidden_size, num_layers, 
    #                  batch_size, learning_rate, lr_decay_step, lr_decay, max_grad):

    # VOCAB_SIZE = 6272  # 词汇表大小

    # SHARE_EMD_WITH_SOFTMAX = True  # 是否在embedding层和softmax层之间共享参数

    # MAX_GRAD = 5.0  # 最大梯度，防止梯度爆炸

    # LEARN_RATE = 0.0005  # 初始学习率

    # LR_DECAY = 0.92  # 学习率衰减

    # LR_DECAY_STEP = 600  # 衰减步数

    # BATCH_SIZE = 64  # batch大小

    # 

    # VOCAB_PATH = 'vocab/poetry.vocab'  # 词表路径

    global_step = model.global_step()
    predict = model.predict()        
    cost = model.loss()
    train_op =  model.optimize()

    saver = tf.train.Saver()

    with tf.Session(graph=g) as sess:
        sess.run(tf.global_variables_initializer())  # 初始化

        for step in range(TRAIN_TIMES):
            # 获取训练batch
            x, y = data.next_batch()


            # 计算loss
            loss, _ = sess.run([cost,train_op],
                               {model.data: x, model.labels: y, model.emb_keep: EMB_KEEP,
                                model.rnn_keep: RNN_KEEP})

            if step % SHOW_STEP == 0:
                print ('step {}, loss is {}'.format(step, loss))
            # 保存模型
            if step % SAVE_STEP == 0:
                saver.save(sess, CKPT_PATH, global_step=model.global_step)


ValueError: Variable rnn/multi_rnn_cell/cell_0/lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-57-0c71523d844e>", line 44, in predict
    outputs, last_state = tf.nn.dynamic_rnn(cell, emb_dropout, dtype=tf.float32, initial_state=self.init_state)
  File "<ipython-input-59-ab13c63ee0eb>", line 46, in <module>
    predict = model.predict()
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
